# Clustering of all data

Read in all necessary packages:

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pandas.io.parsers import read_csv
import scvelo as scv
import scanpy as sc
import numpy as np
from functools import reduce
from anndata import AnnData, read_h5ad
import singlecellmultiomics.bamProcessing.bamToRNACounts
import loompy

In [ ]:
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
sc.set_figure_params(dpi=100, color_map = 'viridis')

# Load the dataset

In [ ]:
pathToData = '/path/to/data'

In [ ]:
#only need to do this once
p1 = pathToData + 'path/to/loom.loom'
p2 = pathToData + 'path/to/loom.loom'
p3 = pathToData + 'path/to/loom.loom'
p4 = pathToData + 'path/to/loom.loom'

paths = [p1,p2,p3,p4]

Here we combine the loom files for all separate libraries into one big loom file ready to import as adata for scanpy. We only need to do this once, afterwards we can just import the .loom directly into adata.

In [ ]:
#only need to do this once
loompy.combine(paths, (pathToData +'dataframes/full_dataset.loom'), key="Accession")

In [ ]:
#import loom in adata
adata = scv.read_loom(pathToData + 'dataframes/full_dataset.loom',obs_names='CellID')
adata

In [ ]:
#check if the library names are okay
adata

In [ ]:
# define a file to write results to
results_file = pathToData + '/dataframes/20210115_Scanpy_scVelo_TCHIC.h5ad'  # the file that will store the analysis results

# Scanpy

## Preprocessing of dataset 

In [ ]:
adata

Check the amount of spliced/unspliced reads in the data.

In [ ]:
scv.pl.proportions(adata)

In [ ]:
#Make names unique
adata.var_names_make_unique()

In [ ]:
#check if variable names are okay
adata.var_names

In [ ]:
adata.var.head()

Show those genes that yield the highest fraction of counts in each single cells, across all cells.

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

Basic filtering. You can play around with these cut-offs to see what works best.

In [ ]:
minCount = 750
sc.pp.filter_cells(adata, min_counts=minCount)
sc.pp.filter_cells(adata, min_genes=100)
sc.pp.filter_genes(adata, min_cells=2)

Let us assemple some information about mitochondrial genes, which are important for quality control.

Citing from "Simple Single Cell" workflows [(Lun, McCarthy & Marioni, 2017)](https://master.bioconductor.org/packages/release/workflows/html/simpleSingleCell.html#examining-gene-level-metrics):

> High proportions are indicative of poor-quality cells (Islam et al. 2014; Ilicic et al. 2016), possibly because of loss of cytoplasmic RNA from perforated cells. The reasoning is that mitochondria are larger than individual transcript molecules and less likely to escape through tears in the cell membrane.

Note you can also use the function `pp.calculate_qc_metrics` to compute the fraction of mitochondrial genes and additional measures.

In [ ]:
mito_genes = adata.var_names.str.startswith('mt-')

# for each cell compute fraction of counts in mito genes vs. all genes
adata.obs['percent_mito'] = np.sum(
    adata[:, mito_genes].X, axis=1) / np.sum(adata.X, axis=1)
# add the total counts per cell as observations-annotation to adata
adata.obs['n_counts'] = adata.X.sum(axis=1)

A violin plot of the computed quality measures.

In [ ]:
sc.pl.violin(adata, ['n_genes', 'n_counts', 'percent_mito'],
             jitter=0.4, multi_panel=True)

Remove cells that have too many mitochondrial genes expressed or too many total counts.

In [ ]:
sc.pl.scatter(adata, x='n_counts', y='percent_mito')
sc.pl.scatter(adata, x='n_counts', y='n_genes')

Actually do the filtering.

In [ ]:
adata = adata[adata.obs['percent_mito'] < 0.7, :]

throw out any other unwanted genes

In [ ]:
# removing genes with annotations we are not interested in
banned = ['NA', 'mt-', "Malat1", 'Rpl', 'Rps']
keptGenes = [geneName for geneName in adata.var.index if not any( (b in geneName for b in banned) )]

adata = adata[:,keptGenes]

adata

minCount = 1000
sc.pp.filter_cells(adata, min_counts=minCount)
sc.pp.filter_cells(adata, min_genes=100)
sc.pp.filter_genes(adata, min_cells=2)

Total-count normalize (library-size correct) the data matrix $\mathbf{X}$ to 10,000 reads per cell, so that counts become comparable among cells.

In [ ]:
sc.pp.normalize_per_cell(adata, counts_per_cell_after=minCount)

Logarithmize the data.

In [ ]:
sc.pp.log1p(adata)

Set the `.raw` attribute of AnnData object to the logarithmized raw gene expression for later use in differential testing and visualizations of gene expression. This simply freezes the state of the AnnData object. While many people consider the normalized data matrix as the "relevant data" for visualization and differential testing, some would prefer to store the unnormalized data.

In [ ]:
adata.raw = adata

Identify highly-variable genes.

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(adata)

Actually do the filtering.

In [ ]:
adata = adata[:, adata.var['highly_variable']]

Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed. Scale the data to unit variance.

In [ ]:
sc.pp.regress_out(adata, ['n_counts', 'percent_mito'])

Scale each gene to unit variance. Clip values exceeding standard deviation 10. 

In [ ]:
sc.pp.scale(adata, max_value=10)

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

## Principal component analysis

Reduce the dimensionality of the data by running principal component analysis (PCA), which reveals the main axes of variation and denoises the data.

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')

We can make a scatter plot in the PCA coordinates, but we will not use that later on.

In [ ]:
plotgene = [geneName for geneName in adata.var.index if any( (b in geneName for b in ['S100g']) )]
sc.pl.pca(adata, color=plotgene, color_map = 'viridis')

In [ ]:
sc.pl.pca(adata, color = ['Muc6', 'Gata6'], color_map = 'viridis')

Let us inspect the contribution of single PCs to the total variance in the data. This gives us information about how many PCs we should consider in order to compute the neighborhood relations of cells, e.g. used in the clustering function  `sc.tl.louvain()` or tSNE `sc.tl.tsne()`. In our experience, often, a rough estimate of the number of PCs does fine.

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

Save the result.

In [ ]:
adata.write(results_file)

## Computing the neighborhood graph

Let us compute the neighborhood graph of cells using the PCA representation of the data matrix. You might simply use default values here. For the sake of reproducing Seurat's results, let's take the following values.

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

## Embedding the neighborhood graph

We advertise embedding the graph in 2 dimensions using UMAP ([McInnes et al., 2018](https://arxiv.org/abs/1802.03426)), see below. It is  potentially more faithful to the global connectivity of the manifold than tSNE, i.e., it better preservers trajectories. In some ocassions, you might still observe disconnected clusters and similar connectivity violations. They can usually be remedied by running:

```
tl.paga(adata)
pl.paga(adata, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
tl.umap(adata, init_pos='paga')
```

In [ ]:
sc.tl.umap(adata)

We can define which characteristics of the data we would like to display. E.g. libraries, histone marks, time points etc. Then we can easily plot those to check for possible batch effects.

In [ ]:
batches = [ob.split(':')[0] for ob in adata.obs_names]
types = [ob.split('-')[1] for ob in batches]
marks = [ob.split('-')[4] for ob in batches]

adata.obs['batch'] = batches 
adata.obs['type'] = types 
adata.obs['mark'] = marks 

In [ ]:
#they are now added under 'obs'
adata

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
sc.pl.umap(adata, color='n_genes', size = 20,ax=ax, color_map = 'viridis')

In [ ]:
sc.pl.umap(adata, color=['batch','type', 'mark'],size=20)

In [ ]:
sc.pl.umap(adata, color=['type', 'mark'],size=20)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sc.pl.umap(adata, color='batch',ax=ax, palette=sns.color_palette("hls", 19),size=20)


As we set the `.raw` attribute of `adata`, the previous plots showed the "raw" (normalized, logarithmized, but uncorrected) gene expression. You can also plot the scaled and corrected gene expression by explicitly stating that you don't want to use `.raw`.

In [ ]:
sc.pl.umap(adata, color=['Lyz1'], use_raw=True, color_map = 'viridis')
sc.pl.umap(adata, color=['Lyz1'], use_raw=False, color_map = 'viridis')

We can now plot all of our favourite genes and check their expression and distribution.

In [ ]:
sc.pl.umap(adata, color=['Lrig1', 'Lgr5', 'Olfm4',
                         'Vil1', 'Muc2', 'Chgb', 'Lyz1'], use_raw=True, color_map = 'viridis')

## Clustering the neighborhood graph

As Seurat and many others, we recommend the Louvain graph-clustering method (community detection based on optimizing modularity). It has been proposed for single-cell data by [Levine et al. (2015)](https://doi.org/10.1016/j.cell.2015.05.047). Note that Louvain clustering directly clusters the neighborhood graph of cells, which we already computed in the previous section.

In [ ]:
sc.tl.leiden(adata)

In [ ]:
sc.tl.louvain(adata)

Plot the clusters, which agree quite well with the result of Seurat.

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sc.pl.umap(adata, color='leiden',ax=ax, palette=sns.color_palette("hls", 23),size=20)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sc.pl.umap(adata, color='louvain',ax=ax, palette=sns.color_palette("hls", 21),size=20)

In [ ]:
sc.pl.umap(adata, color=['louvain','leiden'], palette=sns.color_palette("hls", 23),size=20)

we can save the cells + louvain or leiden clusters in a table, which we can export to a csv and use for e.g. bam splitting.

In [ ]:
cells_clusters = pd.DataFrame(adata.obs['leiden'])
cells_clusters
#cells_clusters.to_csv(PathToData + 'clusters.csv')

### Define cluster identities based on known marker genes

In [ ]:
# if your color map is being annoying, rerun this:
sc.set_figure_params(dpi=100, color_map = 'viridis')

In [ ]:
sc.pl.umap(adata, color=['leiden', 'Lyz1', 'Gata6'], use_raw=True)

Save the result.

In [ ]:
adata.write(results_file)

You can also make a heatmap. ! this takes a bit of time to compute.

In [ ]:
#sc.pl.heatmap(adata, var_names = adata.var_names[adata.var['highly_variable'] & (np.log(adata.var['means'])>-1)])

## Finding marker genes

Let us compute a ranking for the highly differential genes in each cluster. For this, by default, the `.raw` attribute of AnnData is used in case it has been initialized before. The simplest and fastest method to do so is the t-test.

In [ ]:
sc.tl.rank_genes_groups(adata, 'louvain', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
sc.settings.verbosity = 2  # reduce the verbosity

The result of a [Wilcoxon rank-sum (Mann-Whitney-U)](https://de.wikipedia.org/wiki/Wilcoxon-Mann-Whitney-Test) test is very similar. We recommend using the latter in publications, see e.g., [Sonison & Robinson (2018)](https://doi.org/10.1038/nmeth.4612). You might also consider much more powerful differential testing packages like MAST, limma, DESeq2 and, for python, the recent diffxpy.

In [ ]:
sc.tl.rank_genes_groups(adata, 'louvain', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

Save the result.

In [ ]:
adata.write(results_file)

As an alternative, let us rank genes using logistic regression. For instance, this has been suggested by [Natranos et al. (2018)](https://doi.org/10.1101/258566). The essential difference is that here, we use a multi-variate appraoch whereas conventional differential tests are uni-variate. [Clark et al. (2014)](https://doi.org/10.1186/1471-2105-15-79) has more details.

In [ ]:
sc.tl.rank_genes_groups(adata, 'louvain', method='logreg')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

Reload the object that has been save with the Wilcoxon Rank-Sum test result.

In [ ]:
adata = sc.read(results_file)

Show the 10 top ranked genes per cluster 0, 1, ..., 7 in a dataframe.
Columns are louvain clusters.

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(20)

In [ ]:
sc.pl.umap(adata, color='louvain', palette = sns.color_palette("tab20",21),legend_loc='on data')

We can plot the top x differentially expressed genes for each cluster.

In [ ]:
cluster = '14'
number_genes = 10

genes = pd.DataFrame(adata.uns['rank_genes_groups']['names'])[cluster].head(number_genes)
sc.pl.umap(adata, color=genes, color_map = 'viridis')

Get a table with the scores and groups.

In [ ]:
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']}).head(5)

In [ ]:
rank_genes_groups = pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']})

rank_genes_groups.head()

In [ ]:
rank_genes_groups.to_csv(pathToData + 'rank_genes_groups_leidenclusters.csv')

Compare to a single cluster. 

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', groups=['0'], reference='1', method='wilcoxon')
sc.pl.rank_genes_groups(adata, groups=['0'], n_genes=20)

## Various ways of displaying interesting (marker) genes

Reload the object that computed differential expression by comparing to the rest of the groups.

In [ ]:
adata = sc.read(results_file)

Let us also define a list of marker genes for later reference.

In [ ]:
marker_genes = ['Lyz1', 'Vil1', 'Olfm4']

If we want a more detailed view for a certain group, use `sc.pl.rank_genes_groups_violin`.

In [ ]:
sc.pl.rank_genes_groups_violin(adata, groups='0', n_genes=8)

In [ ]:
sc.pl.rank_genes_groups_violin(adata, groups='0', n_genes=8)

If you want to compare a certain gene across groups, use the following.

In [ ]:
sc.pl.violin(adata, marker_genes, groupby='louvain')

Now that we annotated the cell types, let us visualize the marker genes.

In [ ]:
ax = sc.pl.dotplot(adata, marker_genes, groupby='louvain') # ,save='dotplot_markergenes.png')

There is also a very compact violin plot.

In [ ]:
ax = sc.pl.stacked_violin(adata, marker_genes, groupby='louvain', rotation=90)

And a trackplot.

In [ ]:
sc.pl.rank_genes_groups_tracksplot(adata, n_genes=6, key='rank_genes_groups')

# Computing RNA velocity

filter and normalise

In [ ]:
scv.pp.filter_genes(adata, min_shared_counts=10)
scv.pp.normalize_per_cell(adata)
scv.pp.filter_genes_dispersion(adata, n_top_genes=2000)
scv.pp.log1p(adata)

In [ ]:
scv.pp.filter_and_normalize(adata, min_shared_counts=10, n_top_genes=5000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

In [ ]:
scv.tl.velocity(adata)

In [ ]:
scv.tl.velocity_graph(adata)

## Project velocities

We can choose different ways of visualising the velocity onto the umap

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.velocity_embedding_stream(adata, ax=ax, basis='umap', color = 'louvain', palette=sns.color_palette("hls", 21))

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.velocity_embedding(adata, ax=ax, arrow_length=3, arrow_size=2, dpi=120, color = 'louvain', palette=sns.color_palette("hls", 21))

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.velocity_embedding_grid(adata, ax=ax, basis='umap', color = 'louvain', palette=sns.color_palette("hls", 21))

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.velocity_graph(adata, ax=ax, color = 'louvain', palette=sns.color_palette("hls", 21))

The cell cycle detected by RNA velocity, is biologically affirmed by cell cycle scores (standardized scores of mean expression levels of phase marker genes).

In [ ]:
scv.tl.score_genes_cell_cycle(adata)
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.scatter(adata, ax=ax,color_gradients=['S_score', 'G2M_score'], smooth=True, perc=[5, 95])

Two more useful stats: - The speed or rate of differentiation is given by the length of the velocity vector. - The coherence of the vector field (i.e., how a velocity vector correlates with its neighboring velocities) provides a measure of confidence.

In [ ]:
scv.tl.velocity_confidence(adata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(adata, c=keys, cmap='coolwarm', perc=[5, 95])

In [ ]:
df = adata.obs.groupby('leiden')[keys].mean().T
df.style.background_gradient(cmap='coolwarm', axis=1)

We can also calculate and plot the velocity pseudotime

In [ ]:
scv.tl.velocity_pseudotime(adata)
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.scatter(adata, ax=ax, color='velocity_pseudotime', cmap='gnuplot')

To see the different batches on top of the 'velocity' umap, just change the 'color'.

In [ ]:
scv.settings.set_figure_params('scvelo')
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.velocity_embedding_stream(adata, ax=ax,
                                 basis='umap', 
                                 color=['louvain'],palette=sns.color_palette("hls", 21))

In [ ]:
scv.settings.set_figure_params('scvelo')
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.velocity_embedding_stream(adata, ax=ax,
                                 basis='umap', 
                                 color='batch',palette=sc.pl.palettes.default_20, size = 60)

In [ ]:
scv.settings.set_figure_params('scvelo')
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.velocity_embedding_stream(adata, 
                                 basis='umap', ax=ax,
                                 color='type',palette=sc.pl.palettes.default_20)

In [ ]:
scv.settings.set_figure_params('scvelo')
fig, ax = plt.subplots(figsize=(6,5))
scv.pl.velocity_embedding_stream(adata, 
                                 basis='umap', ax=ax,
                                 color='mark',palette=sc.pl.palettes.default_20, size = 60)

Plotting our favourite genes in terms of velocity:

In [ ]:
scv.pl.velocity(adata, var_names=['Lyz1'], ncols=1)

To define the top velocity genes, we need to rank them per cluster.

In [ ]:
scv.tl.rank_velocity_genes(adata, groupby = 'louvain', min_counts = 1, resolution=0.4)

import pandas as pd
pd.DataFrame(adata.uns['rank_velocity_genes']['names']).head(10)



We can then visualise these too.

In [ ]:
scv.pl.velocity(adata, var_names=pd.DataFrame(adata.uns['rank_velocity_genes']['names']).head(5).values.flatten(), ncols=1, colorbar=True)

In [ ]:
sc.pl.umap(adata, color=['initial_size_spliced','initial_size_unspliced','velocity_self_transition'], color_map = 'viridis')

## Finally: we can display our new annotate cell names onto the umap

In [ ]:
#define new cluster names
new_cluster_names = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14']

Actually mark the cell types.

In [ ]:
#rename louvain or leiden clusters to your new defined cell types
adata.rename_categories('louvain', new_cluster_names)

Show off your new umap

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sc.pl.umap(adata, color='louvain',ax=ax, palette=sc.pl.palettes.default_20,legend_loc='on data', title='', frameon=False, size = 100)

During the course of this analysis, the AnnData accumlated the following annotations.

In [ ]:
adata

In [ ]:
adata.write(results_file, compression='gzip')  # `compression='gzip'` saves disk space, but slows down writing and subsequent reading

Get a rough overview of the file using `h5ls`, which has many options - for more details see [here](https://github.com/theislab/scanpy_usage/blob/master/170505_seurat/info_h5ad.md). The file format might still be subject to further optimization in the future. All reading functions will remain backwards-compatible, though.

If you want to share this file with people who merely want to use it for visualization, a simple way to reduce the file size is by removing the dense scaled and corrected data matrix. The file still contains the raw data used in the visualizations.

In [ ]:
adata.X = None
adata.write('./write/Scanpy120hAA.h5ad', compression='gzip')

If you want to export to "csv", you have the following options:

In [ ]:
# Export single fields of the annotation of observations
# adata.obs[['n_counts', 'louvain_groups']].to_csv(
#     './write/pbmc3k_corrected_louvain_groups.csv')

# Export single columns of the multidimensional annotation
# adata.obsm.to_df()[['X_pca1', 'X_pca2']].to_csv(
#     './write/pbmc3k_corrected_X_pca.csv')

# Or export everything except the data using `.write_csvs`.
# Set `skip_data=False` if you also want to export the data.
# adata.write_csvs(results_file[:-5], )